In [ ]:
#METHOD3 for DEG majority_voting

# **Download** Database from Source

# Install **CellphoneDB**

In [1]:
import pandas as pd
import glob
import os

In [2]:
import cellphonedb


# **Display database versions**

In [3]:
from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils

display(HTML(db_releases_utils.get_remote_database_versions_html()['db_releases_html_table']))


# **Define the version and the path to download database**

In [5]:
# -- Version of the databse
cpdb_version = 'v5.0.0'

# -- Path where the input files to generate the database are located
cpdb_target_dir = os.path.join('/home/icb/shahana.dilruba/cpdb_m3_m_voting/', cpdb_version)

# Download **database**

In [6]:
from cellphonedb.utils import db_utils

db_utils.download_database(cpdb_target_dir, cpdb_version)

Downloaded cellphonedb.zip into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0
Downloaded complex_input.csv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0
Downloaded gene_input.csv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0
Downloaded interaction_input.csv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0
Downloaded protein_input.csv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0
Downloaded uniprot_synonyms.tsv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/sources
Downloaded transcription_factor_input.csv into /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/sources



# **Build database from files**

In [7]:
# -- Path where the input files to generate the database are located
cpdb_input_dir = '/home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0'
os.listdir(cpdb_input_dir)

['gene_input.csv',
 'sources',
 'interaction_input.csv',
 'protein_input.csv',
 'complex_input.csv',
 'cellphonedb.zip']

# **Generate database**

CREATE DATABASE

In [8]:
from cellphonedb.utils import db_utils
# -- Creates new database
db_utils.create_db(cpdb_input_dir)




Created /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/cellphonedb_03_27_2024_090855.zip successfully


In [9]:
mv /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/cellphonedb_03_27_2024_090855.zip /home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/cellphonedb.zip


**METHOD 3**

# Input **files**

In [19]:
cpdb_file_path = '/home/icb/shahana.dilruba/cpdb_m3_m_voting/v5.0.0/cellphonedb.zip'
meta_file_path = '/home/icb/shahana.dilruba/deg_tsv/lungatlas_model_chosen_meta.tsv'
counts_file_path = '/home/icb/shahana.dilruba/celltypist_model_chosen_lungatlas_data.h5ad'
degs_file_path = '/home/icb/shahana.dilruba/deg_tsv/differential_expression_results_majority_voting.tsv'
out_path = '/home/icb/shahana.dilruba/cpdb_m3_m_voting/results'

In [20]:
import pandas as pd


# Inspect input **files**

In [21]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

,Cell,majority_voting
0,muc21058_AAACCCAAGTTTGAGA,Club (non-nasal)
1,muc21058_AAACCCACACGCGCAT,Club (non-nasal)
2,muc21058_AAACCCACAGACACAG,Suprabasal


# **The counts files is a scanpy h5ad object. The dimensions and order of this object must coincide with the dimensions of the metadata file (i.e. must have the same number of cells in both files)**


In [22]:
import anndata

In [23]:
adata = anndata.read_h5ad(counts_file_path)
adata.shape

(254630, 18115)

# **Check barcodes in metadata and counts are the same.**


In [25]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()

True

In [26]:
#Differentially expressed genes
pd.read_csv(degs_file_path, sep = '\t') \
    .head(3)

,Group,Gene,Log Fold Change,P-value,Adjusted P-value
0,?,TMSB4X,2.031221,0.0,0.0
1,?,CDKN2A,3.059751,0.0,0.0
2,?,IL32,4.776148,0.0,0.0


# No microenvironment data included in this analysis


# 
Run CellphoneDB with differential analysis (method 3) with DEG for majority voting**

In [27]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method

cpdb_results = cpdb_degs_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                            # mandatory: CellphoneDB database zip file.
    meta_file_path = meta_file_path,                            # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,                        # mandatory: normalized count matrix - a path to the counts file, or an in-memory AnnData object
    degs_file_path = degs_file_path,                            # mandatory: tsv file with DEG to account.
    counts_data = 'hgnc_symbol',                                # defines the gene annotation in counts matrix.   
    score_interactions = True,                                  # optional: whether to score interactions or not. 
    threshold = 0.1,                                            # defines the min % of cells expressing a gene for this to be employed in the analysis.
    result_precision = 3,                                       # Sets the rounding for the mean values in significan_means.
    separator = '|',                                            # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                              # Saves all intermediate tables emplyed during the analysis in pkl format.
    output_path = out_path,                                     # Path to save results
    output_suffix = None,                                       # Replaces the timestamp in the output files by a user defined string in the  (default: None)
    threads = 25
    )

[ ][CORE][27/03/24-09:37:03][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/home/icb/shahana.dilruba/celltypist_model_chosen_lungatlas_data.h5ad
/home/icb/shahana.dilruba/deg_tsv/lungatlas_model_chosen_meta.tsv
/home/icb/shahana.dilruba/deg_tsv/differential_expression_results_majority_voting.tsv
[ ][CORE][27/03/24-09:37:54][INFO] Running Real Analysis
[ ][CORE][27/03/24-09:37:55][INFO] Running DEGs-based Analysis
[ ][CORE][27/03/24-09:37:56][INFO] Building results
[ ][CORE][27/03/24-09:37:56][INFO] Scoring interactions: Filtering genes per cell type..


100%|███████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  3.41it/s]

[ ][CORE][27/03/24-09:38:01][INFO] Scoring interactions: Calculating mean expression of each gene per group/cell type..



100%|███████████████████████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00, 11.50it/s]
/home/icb/shahana.dilruba/miniforge3/envs/pertpy/lib/python3.10/site-packages/cellphonedb/utils/scoring_utils.py:138: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  matrix[index_name].replace(to_replace=id2name, inplace=True)


[ ][CORE][27/03/24-09:38:03][INFO] Scoring interactions: Calculating scores for all interactions and cell types..


100%|█████████████████████████████████████████████████████████████████████████████████| 196/196 [02:11<00:00,  1.49it/s]


Saved deconvoluted to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_deconvoluted_03_27_2024_094017.txt
Saved deconvoluted_percents to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_deconvoluted_percents_03_27_2024_094017.txt
Saved means to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_means_03_27_2024_094017.txt
Saved relevant_interactions to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_relevant_interactions_03_27_2024_094017.txt
Saved significant_means to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_significant_means_03_27_2024_094017.txt
Saved interaction_scores to /home/icb/shahana.dilruba/cpdb_m3_m_voting/results/degs_analysis_interaction_scores_03_27_2024_094017.txt


In [28]:
#Results are save as files and as a dictionary in the cpdb_results variable.
list(cpdb_results.keys())

['deconvoluted',
 'deconvoluted_percents',
 'means',
 'relevant_interactions',
 'significant_means',
 'CellSign_active_interactions',
 'CellSign_active_interactions_deconvoluted',
 'interaction_scores']

In [29]:
#Description of output files
#Relevant interaction field
cpdb_results['relevant_interactions'].head(2)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,...,viral|Deuterosomal,viral|Goblet (nasal),viral|Hillock-like,viral|Ionocyte,viral|Multiciliated (non-nasal),viral|Neuroendocrine,viral|Suprabasal,viral|pre-TB secretory,viral|tuft,viral|viral
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,...,1,1,1,1,1,1,1,1,1,1
3,CPI-SS0EE09A201,CDH2_CDH2,simple:P19022,simple:P19022,CDH2,CDH2,False,False,False,curated,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#Significant means field
cpdb_results['significant_means'].head(2)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,...,viral|Deuterosomal,viral|Goblet (nasal),viral|Hillock-like,viral|Ionocyte,viral|Multiciliated (non-nasal),viral|Neuroendocrine,viral|Suprabasal,viral|pre-TB secretory,viral|tuft,viral|viral
2531,CPI-SC056F0C465,WNT5B_FZD1_LRP6,simple:Q9H1J7,complex:FZD1_LRP6,WNT5B,NaN,True,False,False,curated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2530,CPI-SC0F416B44F,WNT5B_FZD1_LRP5,simple:Q9H1J7,complex:FZD1_LRP5,WNT5B,NaN,True,False,False,curated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
cpdb_results['interaction_scores'].head(2)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,...,viral|Deuterosomal,viral|Goblet (nasal),viral|Hillock-like,viral|Ionocyte,viral|Multiciliated (non-nasal),viral|Neuroendocrine,viral|Suprabasal,viral|pre-TB secretory,viral|tuft,viral|viral
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,...,3.944,5.847,16.926,2.871,2.988,0.0,7.249,3.94,2.618,5.59
1,CPI-SC09D882C18,CDH1_integrin_aEb7_complex,simple:P12830,complex:integrin_aEb7_complex,CDH1,NaN,False,False,False,curated,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000,0.00


In [32]:
#Means field
cpdb_results['means'].head(2)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,...,viral|Deuterosomal,viral|Goblet (nasal),viral|Hillock-like,viral|Ionocyte,viral|Multiciliated (non-nasal),viral|Neuroendocrine,viral|Suprabasal,viral|pre-TB secretory,viral|tuft,viral|viral
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,...,0.392,0.412,0.618,0.359,0.428,0.279,0.528,0.383,0.347,0.456
1,CPI-SC09D882C18,CDH1_integrin_aEb7_complex,simple:P12830,complex:integrin_aEb7_complex,CDH1,NaN,False,False,False,curated,...,0.200,0.201,0.202,0.000,0.200,0.208,0.202,0.202,0.202,0.201


In [33]:
#deconvoluted fields
cpdb_results['deconvoluted'].head(2)

,gene_name,uniprot,is_complex,protein_name,complex_name,id_cp_interaction,gene,?,Basal resting,Club (nasal),...,Deuterosomal,Goblet (nasal),Hillock-like,Ionocyte,Multiciliated (non-nasal),Neuroendocrine,Suprabasal,pre-TB secretory,tuft,viral
multidata_id,,,,,,,,,,,,,,,,,,,,,
1355,UBASH3B,Q8TF42,True,UBS3B_HUMAN,Dehydroepiandrosterone_bySTS,CPI-CS09B8977D7,UBASH3B,0.016,0.006,0.007,...,0.006,0.012,0.012,0.01,0.01,0.0,0.011,0.008,0.003,0.007
1355,UBASH3B,Q8TF42,True,UBS3B_HUMAN,Dehydroepiandrosterone_bySTS,CPI-CS05760BB78,UBASH3B,0.016,0.006,0.007,...,0.006,0.012,0.012,0.01,0.01,0.0,0.011,0.008,0.003,0.007


In [34]:
# Interaction scores field
cpdb_results['interaction_scores'].head(2)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,...,viral|Deuterosomal,viral|Goblet (nasal),viral|Hillock-like,viral|Ionocyte,viral|Multiciliated (non-nasal),viral|Neuroendocrine,viral|Suprabasal,viral|pre-TB secretory,viral|tuft,viral|viral
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,...,3.944,5.847,16.926,2.871,2.988,0.0,7.249,3.94,2.618,5.59
1,CPI-SC09D882C18,CDH1_integrin_aEb7_complex,simple:P12830,complex:integrin_aEb7_complex,CDH1,NaN,False,False,False,curated,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000,0.00
